In [1]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.4     ✔ purrr   0.3.4
✔ tibble  3.1.2     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



[1] "sliced-s01e10-playoffs-2"

In [2]:
# bring in some packages
library(tidymodels)
library(lubridate)

── Attaching packages ────────────────────────────────────── tidymodels 0.1.3 ──

✔ broom        0.7.7      ✔ rsample      0.1.0 
✔ dials        0.0.9      ✔ tune         0.1.5 
✔ infer        0.5.4      ✔ workflows    0.2.2 
✔ modeldata    0.1.0      ✔ workflowsets 0.0.2 
✔ parsnip      0.1.5      ✔ yardstick    0.0.8 
✔ recipes      0.1.16     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks readr::spec()
✖ recipes::step()   masks stats::step()
• Use tidymodels_prefer() to resolve common conflicts.


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




In [3]:
# read the files
list.files(path = "../input/sliced-s01e10-playoffs-2")
sample_submission <- read_csv('../input/sliced-s01e10-playoffs-2/sample_submission.csv')
train_raw <- read_csv('../input/sliced-s01e10-playoffs-2/train.csv')
test_raw <- read_csv('../input/sliced-s01e10-playoffs-2/test.csv')

[1] "sample_submission.csv" "test.csv"              "train.csv"


── Column specification ────────────────────────────────────────────────────────
cols(
  id = col_double(),
  adoption = col_double(),
  `no outcome` = col_double(),
  transfer = col_double()
)



── Column specification ────────────────────────────────────────────────────────
cols(
  id = col_double(),
  age_upon_outcome = col_character(),
  animal_type = col_character(),
  breed = col_character(),
  color = col_character(),
  date_of_birth = col_date(format = ""),
  datetime = col_datetime(format = ""),
  name = col_character(),
  outcome_type = col_character(),
  sex = col_character(),
  spay_neuter = col_character()
)



── Column specification ────────────────────────────────────────────────────────
cols(
  id = col_double(),
  age_upon_outcome = col_character(),
  animal_type = col_character(),
  breed = col_character(),
  color = col_character(),
  date_of_birth = col_date(format = ""),
  datetime = col_datetime(format = ""),
  name = col_character(),
  sex = col_character(),
 

In [4]:
# EDA
summary(train_raw)

       id        age_upon_outcome   animal_type           breed          
 Min.   :    1   Length:54408       Length:54408       Length:54408      
 1st Qu.:19483   Class :character   Class :character   Class :character  
 Median :38850   Mode  :character   Mode  :character   Mode  :character  
 Mean   :38845                                                           
 3rd Qu.:58277                                                           
 Max.   :77725                                                           
    color           date_of_birth           datetime                  
 Length:54408       Min.   :1994-01-25   Min.   :2013-10-01 09:31:00  
 Class :character   1st Qu.:2012-08-15   1st Qu.:2014-10-18 16:20:30  
 Mode  :character   Median :2014-05-12   Median :2015-11-07 16:14:00  
                    Mean   :2013-08-24   Mean   :2015-11-23 14:39:07  
                    3rd Qu.:2015-09-17   3rd Qu.:2016-12-13 16:24:00  
                    Max.   :2017-12-25   Max.   :2018-02

In [5]:
head(train_raw)

id,age_upon_outcome,animal_type,breed,color,date_of_birth,datetime,name,outcome_type,sex,spay_neuter
<dbl>,<chr>,<chr>,<chr>,<chr>,<date>,<dttm>,<chr>,<chr>,<chr>,<chr>
1265,2 years,Cat,Russian Blue Mix,Blue,2014-04-21,2016-08-05 14:15:00,Smokey,adoption,Male,Fixed
24053,1 year,Other,Bat Mix,Brown,2016-10-24,2017-10-25 08:02:00,NA,no outcome,Unknown,Unknown
4785,2 months,Dog,Chihuahua Shorthair/Pomeranian,Brown,2014-01-04,2014-03-08 16:37:00,NA,transfer,Female,Fixed
65439,2 years,Cat,Domestic Shorthair Mix,Brown Tabby/White,2012-11-26,2014-12-04 12:21:00,Momma Kitty,transfer,Female,Fixed
45732,1 year,Dog,Rat Terrier Mix,White/Chocolate,2014-07-10,2016-01-16 16:46:00,Estrella,transfer,Female,Fixed
38636,2 years,Dog,Beagle/Chihuahua Shorthair,Tricolor,2012-08-30,2014-09-05 13:44:00,Garrison,adoption,Male,Fixed


In [6]:
# count animal types
train_raw %>%
    count(animal_type, sort = TRUE)

animal_type,n
<chr>,<int>
Dog,30830
Cat,20561
Other,2800
Bird,212
Livestock,5


In [7]:
# count breeds
train <- train_raw %>%
    mutate(breed = fct_lump(breed,100)) %>%
    mutate(age = time_length(difftime(as.Date(datetime), as.Date(date_of_birth)),"years")) %>%
    select(id, age, animal_type, breed, color, datetime, sex, spay_neuter, outcome_type) %>%
mutate_if(is.character, factor)

In [8]:
head(train)

id,age,animal_type,breed,color,datetime,sex,spay_neuter,outcome_type
<dbl>,<dbl>,<fct>,<fct>,<fct>,<dttm>,<fct>,<fct>,<fct>
1265,2.2915811,Cat,Other,Blue,2016-08-05 14:15:00,Male,Fixed,adoption
24053,1.0020534,Other,Bat Mix,Brown,2017-10-25 08:02:00,Unknown,Unknown,no outcome
4785,0.1724846,Dog,Other,Brown,2014-03-08 16:37:00,Female,Fixed,transfer
65439,2.0205339,Cat,Domestic Shorthair Mix,Brown Tabby/White,2014-12-04 12:21:00,Female,Fixed,transfer
45732,1.5195072,Dog,Rat Terrier Mix,White/Chocolate,2016-01-16 16:46:00,Female,Fixed,transfer
38636,2.0150582,Dog,Other,Tricolor,2014-09-05 13:44:00,Male,Fixed,adoption


In [ ]:
# clean the data
train <- train_raw %>%
  mutate(breed = fct_lump(breed,100),
         color = fct_lump(breed,50)) %>%
  mutate(age = time_length(difftime(as.Date(datetime), as.Date(date_of_birth)),"years")) %>%
  select(id, age, animal_type, breed, color, datetime, sex, spay_neuter, outcome_type) %>%
  mutate_if(is.character, factor)

test <- test_raw %>%
  mutate(breed = fct_lump(breed,100),
         color = fct_lump(breed,50)) %>%
  mutate(age = time_length(difftime(as.Date(datetime), as.Date(date_of_birth)),"years")) %>%
  select(id, age, animal_type, breed, color, datetime, sex, spay_neuter) %>%
  mutate_if(is.character, factor)

# build a model
set.seed(123)
animal_folds <- vfold_cv(train, v = 5, strata = outcome_type)
animal_folds

library(themis)
library(baguette)

animal_metrics <- metric_set(mn_log_loss, accuracy, sensitivity, specificity)

animal_rec <- recipe(outcome_type ~ ., data = train) %>%
  update_role(id, new_role = "Id") %>%
  step_other(breed) %>%
  step_other(color) %>%
  step_date(datetime) %>%
  step_rm(datetime)

bag_spec <-
  bag_tree(min_n = 10) %>%
  set_engine("rpart", times = 25) %>%
  set_mode("classification")

animal_wf <-
  workflow() %>%
  add_recipe(animal_rec) %>%
  add_model(bag_spec)

animal_res <- fit_resamples(
  animal_wf,
  resamples = animal_folds,
  metrics = animal_metrics,
  control = control_resamples(save_pred = TRUE)
)

collect_metrics(animal_res)  

animal_res %>%
  collect_predictions() %>%
  conf_mat(outcome_type, .pred_class)  

set.seed(123)
bag_fit <- fit(animal_wf, data = train)
bag_fit

test_rs <- augment(bag_fit, test)

prediction <- test_rs %>%
  select(id, .pred_adoption, `.pred_no outcome`, .pred_transfer)

colnames(prediction) <- c("id", "adoption", "no outcome", "transfer")  
  
write_csv(prediction, 'my_prediction.csv')


.metric,.estimator,mean,n,std_err,.config
<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
accuracy,multiclass,0.8034297,5,0.0006728516,Preprocessor1_Model1
mn_log_loss,multiclass,0.5192446,5,0.0030651386,Preprocessor1_Model1
sens,macro,0.6912061,5,0.0019864050,Preprocessor1_Model1
spec,macro,0.8576288,5,0.0005930102,Preprocessor1_Model1


            Truth
Prediction   adoption no outcome transfer
  adoption      31133       1128     5958
  no outcome      180       2468      328
  transfer       1962       1139    10112